In [2]:
from ai.train import TrainNNConfig
%load_ext autoreload
%autoreload 2
config = TrainNNConfig.from_yaml_file('/Users/egafni/projects/AI/experiments/next_token/wavenet/exp_0/train.config')

In [19]:
dm = config.datamodule.i()
dm.prepare_data()

2023-11-25 00:20:06.572 | INFO     | ai.datamodule.text:prepare_data:76 - Creating datasets...


In [20]:
)

In [7]:
model_class = config.lightning_module.get_target()
model = model_class.load_from_checkpoint('/Users/egafni/projects/AI/experiments/next_token/wavenet/exp_0/checkpoints/epoch0__step400.ckpt')

In [12]:
x,y = next(iter(dm.test_dataloader()))

In [23]:
# dm.train_dataloader.dat
y_hat = model.generate(x[:1,:], 100)


In [28]:
x[:1,:]

tensor([[14, 21,  5, 12,  0,  1, 14, 19, 21, 13,  1, 14,  1,  0,  1, 14, 20,  1,
         18,  5, 19,  0,  1, 14, 21, 19,  8,  0,  1, 15,  0,  1]])

In [24]:
y_hat

tensor([[13,  2,  5, 19, 20,  5, 25,  1,  5,  0, 10,  1, 12,  9, 12, 14, 25,  0,
         16,  1, 13,  1,  0, 10,  1,  0, 11,  5, 12,  1, 14,  0, 22,  9,  5, 12,
         25, 14,  0, 13,  1, 13, 25,  9,  5,  0, 17, 11,  9, 19, 19,  5,  0, 10,
         15, 13, 25,  0, 19,  5, 14, 26,  1,  9,  0, 11,  9, 22,  9,  0, 26,  5,
         12,  5, 18, 25,  1,  8,  0, 14,  1, 13, 25,  0,  3,  1, 12,  9,  1,  0,
         18,  1, 12,  5,  5, 12, 12,  1,  0, 19]])

In [27]:
print(''.join(dm.ds_train.decode(y_hat[0])))

mbesteyae
jalilny
pama
ja
kelan
vielyn
mamyie
qkisse
jomy
senzai
kivi
zeleryah
namy
calia
raleella
s


In [ ]:
class WaveBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dilation_rates, kernel_size):
        super(WaveBlock, self).__init__()
        self.num_rates = dilation_rates
        self.convs = nn.ModuleList()
        self.filter_convs = nn.ModuleList()
        self.gate_convs = nn.ModuleList()

        self.convs.append(nn.Conv1d(in_channels, out_channels, kernel_size=1))
        dilation_rates = [2 ** i for i in range(dilation_rates)]
        for dilation_rate in dilation_rates:
            self.filter_convs.append(
                nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size,
                          padding=int((dilation_rate * (kernel_size - 1)) / 2), dilation=dilation_rate))
            self.gate_convs.append(
                nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size,
                          padding=int((dilation_rate * (kernel_size - 1)) / 2), dilation=dilation_rate))
            self.convs.append(nn.Conv1d(out_channels, out_channels, kernel_size=1))

    def forward(self, x):
        x = self.convs[0](x)
        res = x
        for i in range(self.num_rates):
            x = torch.tanh(self.filter_convs[i](x)) * torch.sigmoid(self.gate_convs[i](x))
            x = self.convs[i + 1](x)
            res = res + x
        return res